In [87]:
import numpy as np
import pandas as pd
from collections import defaultdict

import os
import sys

from sklearnex import patch_sklearn
patch_sklearn()

import sklearn

import sklearn.pipeline 
import sklearn.model_selection
import sklearn.metrics
import sklearn.linear_model

from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool

import optuna

sys.path.append("/data/ouga/home/ag_gagneur/l_vilov/workspace/species-aware-DNA-LM/mpra_griesemer/utils") 

from models import *
from misc import dotdict

import scipy.stats
import pickle

import gensim.models 

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
data_dir = '/s/project/mll/sergey/effect_prediction/MLM/agarwal_2022/'

In [3]:
input_params = dotdict({})

input_params.model = 'MLM' #embedding name, can be "MLM" "word2vec" "griesemer" or "Nmers" where N is an integer

input_params.output_dir = './test' #output folder

input_params.N_trials = 1000 #number of optuna trials
input_params.keep_first = True #perform hpp search only at the first split, then use these hyperparameters

input_params.N_splits = 100 #number of GroupShuffleSplits
input_params.N_CVsplits = 5 #number of CV splits for hyperparameter search
input_params.seed = 1 #seed fot GroupShuffleSplit

In [35]:
#get MLM embeddings

with open(data_dir + '../species_aware_emb/all_3utr.pickle', 'rb') as f:
            mlm_embeddings = pickle.load(f)
            utr_names =  pickle.load(f)

In [97]:
#MLM embeddings are made for transcripts
#get corresponding gene names

embedding_transcripts = [x.split('.')[0] for x in utr_names]

transcript_to_gene = pd.read_csv(data_dir + '../UTR_coords/GRCh38_EnsembleCanonical_HGNC.tsv.gz', sep='\t', 
                                     names=['gene_id','transcript_id'], skiprows=1,usecols=[0,1]).set_index('transcript_id')

In [109]:
#get FASTA seqs

human_fasta = data_dir + '../fasta/240_mammals/species/Homo_sapiens.fa'

human_utr = defaultdict(str)

with open(human_fasta, 'r') as f:
    for line in f:
        if line.startswith('>'):
            transcript_id = line[1:].split(':')[0].split('.')[0]
        else:
            human_utr[transcript_id] += line.rstrip().upper()

In [115]:
utr_df = pd.DataFrame(human_utr.values(),
             index=transcript_to_gene.loc[human_utr.keys()].gene_id, 
             columns=['seq'])

In [151]:
folds_df = pd.read_csv(data_dir + 'saluki_paper/Fig3_S4/binnedgenes.txt', sep='\t', usecols=[0,1],
                      names=['Fold','gene_id'], skiprows=1).set_index('gene_id') #folds as they are in Agarwal article

In [153]:
data_df = [folds_df]

df = pd.read_csv(data_dir + 'human/seqFeatWithKmerFreqs.txt.gz', sep='\t', 
                          usecols=lambda x: not 'ORF.' in x and not 'UTR.' in x).set_index('GENE') #basic features (8) + codons (62)

data_df.append(df)

In [154]:
#SeqWeaver RBP binding (780)
for region in ('3pUTR','5pUTR','ORF'):
    df = pd.read_csv(data_dir + f'human/SeqWeaver_predictions/{region}_avg.txt.gz', sep='\t').set_index('Group.1')
    data_df.append(df)

In [155]:
#miRNA target repression (319)
df = pd.read_csv(data_dir + f'human/CWCS.txt.gz', sep='\t').set_index('GeneID')
data_df.append(df)

In [156]:
data_df = pd.concat(data_df,axis=1) #concat all features, except embeddings

In [157]:
data_df = data_df[~data_df.HALFLIFE.isna()]
data_df.fillna(0, inplace=True)

In [158]:
#get sequence embedding depending on the model

if input_params.model=='MLM':
            

    embeddings_df = pd.DataFrame(mlm_embeddings, 
                                     index=transcript_to_gene.loc[embedding_transcripts].gene_id, 
                                     columns=[f'emb_{x}' for x in range(mlm_embeddings.shape[1])])

elif 'mers' in input_params.model:
    
    k = int(input_params.model[0])
        
    kmerizer = Kmerizer(k=k)
    
    Nmer_embeddings = utr_df.seq.apply(lambda x: kmerizer.kmerize(x))
    
    embeddings_df = pd.DataFrame(Nmer_embeddings.tolist(), index=Nmer_embeddings.index, columns=[f'emb_{x}' for x in range(4**k)])

elif input_params.model=='word2vec':
        
    kmerizer_w2v = Kmerizer(k=4)

    w2v_model = gensim.models.Word2Vec(sentences=utr_df.seq.apply(lambda x: kmerizer_w2v.tokenize(x)), 
                             vector_size=128, window=5, min_count=1, workers=4, sg=1) #default: CBOW

    word2vec_emb = utr_df.seq.apply(
        lambda x: np.mean([w2v_model.wv[x]  for x in kmerizer_w2v.tokenize(x)],axis=0)) #average embedding of all 4-mers in the sequence

    word2vec_emb = word2vec_emb[~word2vec_emb.isna()]
    
    embeddings_df = pd.DataFrame(word2vec_emb.tolist(), index=word2vec_emb.index, columns=[f'emb_{x}' for x in range(128)])
    
data_df = pd.concat([data_df,embeddings_df], join='inner', axis=1)

X = data_df.iloc[:,2:].values #all columns except HALFLIFE and fold
y = data_df['HALFLIFE'].values
folds = data_df['Fold'].values

In [ ]:
def apply_SVR(args):
        
    fold_idx, test_hpp = args 

    test_idx = mpra_df[mpra_df.Fold==str(fold_idx)].index
    train_idx = mpra_df[(mpra_df.Fold!=str(fold_idx))&(mpra_df.Fold!='Test')].index

    pipe = sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(),
                                                  sklearn.svm.SVR(**test_hpp))
    pipe.fit(X[train_idx],y[train_idx])

    R2_score = pipe.score(X[test_idx],y[test_idx])
        
    return R2_score


def hpp_search(X,y, mpra_df, cv_splits = 10):
    
    '''
    Perform Hyperparameter Search using OPTUNA Bayesian Optimisation strategy
    
    The bets hyperparameters should maximize coefficient of determination (R2)
    
    The hyperparameter range should first be adjused with grid search to make the BO algorithm converge in reasonable time
    '''


    def objective(trial):

        C = trial.suggest_float("C", 1e-4, 1e4, log=True)
        epsilon = trial.suggest_float("epsilon", 1e-5, 1, log=True)
        gamma = trial.suggest_float("gamma", 1e-5, 1, log=True)

        test_hpp = {'C':C, 'epsilon':epsilon, 'gamma':gamma}
        
        pool = Pool(processes=input_params.n_jobs,maxtasksperchild=5)

        cv_scores = []
        
        params = ((fold_idx, test_hpp) for fold_idx in range(cv_splits))
        
        for res in pool.imap(apply_SVR,params):
            cv_scores.append(res)
     
        pool.close()
        pool.join()
    
        return np.mean(cv_scores)
    
    study = optuna.create_study(direction = "maximize")

    study.optimize(objective, n_trials = input_params.N_trials)
    
    best_params = study.best_params
    
    return best_params

In [ ]:
#cv_res = np.zeros((input_params.N_splits,len(y))) #predictions for each point in each split
cv_res = np.zeros((len(set(groups)),len(y))) #predictions for each point in each split
cv_res[:] = np.NaN 

#cv_res_lasso = np.zeros((input_params.N_splits,len(y))) #predictions for each point in each split
cv_res_lasso = np.zeros((len(set(groups)),len(y))) #predictions for each point in each split
cv_res_lasso[:] = np.NaN 

cv_scores = [] #scores and best hyperparameters for each split

#gss = sklearn.model_selection.LeaveOneGroupOut() 

input_params.N_splits=10

gss = sklearn.model_selection.LeaveOneGroupOut() 

best_hpp = {'C': 0.12227907014412719, 'epsilon': 0.002017028863020407, 'gamma': 0.0017418739573905068}#MLM
best_hpp = {'C': 0.06074046127798098, 'epsilon': 0.032758360822165884, 'gamma': 0.0005466435051403621} #5mers

#best_hpp = {'alpha':1e-5}

for round_idx, (train_idx, test_idx) in enumerate(gss.split(X, y, groups)):
        
        X_train, X_test, y_train, y_test = X[train_idx,:],X[test_idx,:],y[train_idx],y[test_idx]
        
        if round_idx==0 or input_params.keep_first==False:
            #perform only ones if input_params.keep_first==True
            best_hpp = hpp_search(X_train,y_train,groups[train_idx],cv_splits = input_params.N_CVsplits)
        
        pipe = sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(),
                                              sklearn.svm.SVR(**best_hpp))
        
        #pipe = sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(),
        #                                      sklearn.linear_model.Lasso(**best_hpp))
        
        pipe.fit(X_train,y_train)
                    
        y_pred = pipe.predict(X_test) 
        
        cv_res[round_idx,test_idx] = y_pred
        
        group_kfold = sklearn.model_selection.GroupKFold(n_splits=input_params.N_CVsplits).split(X_train,y_train,groups[train_idx])
        pipe_lasso = sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(), sklearn.linear_model.LassoCV(cv=group_kfold, alphas=10.**np.arange(-6,0))) 
        pipe_lasso.fit(X_train,y_train)
        y_pred_lasso = pipe_lasso.predict(X_test)
        
        cv_res_lasso[round_idx,test_idx] = y_pred_lasso

        cv_scores.append({'round':round_idx,
                          'chrom':groups[test_idx][0],
                         'r2_svr':sklearn.metrics.r2_score(y_test,y_pred),
                         'pearson_r_svr':scipy.stats.pearsonr(y_test,y_pred)[0],
                         'r2_lasso':sklearn.metrics.r2_score(y_test,y_pred_lasso),
                         'pearson_r_lasso':scipy.stats.pearsonr(y_test,y_pred_lasso)[0]
                         }|best_hpp)
        
cv_scores = pd.DataFrame(cv_scores)

In [492]:
cv_scores.pearson_r_svr.mean()

0.2038695562423924

In [426]:
from sklearn.model_selection import cross_val_score

In [427]:
def pearson_r(estimator, X, y):
    y_pred = estimator.predict(X)
    #print(estimator[1].alpha_)
    if len(y_pred.shape) == 2:
        y_pred = y_pred.reshape(-1)
    return scipy.stats.pearsonr(y, y_pred)[0]

In [ ]:
pipe = sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(), sklearn.linear_model.LassoCV(cv=3, alphas=10.**np.arange(-6,0))) 
#limit runtime
r2 = cross_val_score(pipe,X[train_idx],y[train_idx],scoring=pearson_r,
                     cv=sklearn.model_selection.GroupKFold(n_splits=10), groups=groups[train_idx], n_jobs=-1)#.mean()

In [472]:
np.mean(r2)

0.496721636952289

In [ ]:
os.makedirs(input_params.output_dir, exist_ok=True) #make output dir

cv_scores.to_csv(input_params.output_dir + '/cv_scores.tsv', sep='\t', index=None) #save scores

with open(input_params.output_dir + '/cv_res.npy', 'wb') as f:
    np.save(f, cv_res) #save predictions at each round